# IntegratedML applied to biomedical data, using PyODBC
This notebook demonstrates the following:
- Connecting to InterSystems IRIS via PyODBC connection
- Creating, Training and Executing (PREDICT() function) an IntegratedML machine learning model, applied to breast cancer tumor diagnoses
- INSERTING machine learning predictions into a new SQL table
- Executing a relatively complex SQL query containing IntegratedML PREDICT() and PROBABILITY() functions, and flexibly using the results to filter and sort the output

### ODBC and pyODBC Resources
Often, connecting to a database is more than half the battle when developing SQL-heavy applications, especially if you are not familiar with the tools, or more importantly the particular database system. If this is the case, and you are just getting started using PyODBC and InterSystems IRIS, this notebook and these resources below may help you get up to speed!

https://gettingstarted.intersystems.com/development-setup/odbc-connections/

https://irisdocs.intersystems.com/irislatest/csp/docbook/DocBook.UI.Page.cls?KEY=BNETODBC_support#BNETODBC_support_pyodbc

https://stackoverflow.com/questions/46405777/connect-docker-python-to-sql-server-with-pyodbc

https://stackoverflow.com/questions/44527452/cant-open-lib-odbc-driver-13-for-sql-server-sym-linking-issue

In [1]:
# make the notebook full screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### 1. Install system packages for ODBC

In [2]:
!apt-get update
!apt-get install gcc
!apt-get install -y tdsodbc unixodbc-dev
!apt install unixodbc-bin -y
!apt-get clean 

Get:1 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]             
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]    
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [1161 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic/multiverse amd64 Packages [186 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic/main amd64 Packages [1344 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic/restricted amd64 Packages [13.5 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic/universe amd64 Packages [11.3 MB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [22.8 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2990 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/multiverse amd64 Packages [29.9 kB]
Get:

Get:7 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxext6 amd64 2:1.3.3-1 [29.4 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic/main amd64 fontconfig amd64 2.12.6-0ubuntu2 [169 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmng2 amd64 2.0.2-0ubuntu3 [169 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 x11-common all 1:7.7+19ubuntu7.1 [22.5 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic/main amd64 libice6 amd64 2:1.0.9-2 [40.2 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsm6 amd64 2:1.2.2-1 [15.8 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxt6 amd64 1:1.1.5-1 [160 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic/main amd64 libaudio2 amd64 1.9.4-6 [50.3 kB]
Get:15 http://archive.ubuntu.com/ubuntu bionic/main amd64 mysql-common all 5.8+1.0.4 [7308 B]
Get:16 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmysqlclient20 amd64 5.7.39-0ubuntu0.18.04.2 [728 kB]
Get:17 http://archiv

7Progress: [ 24%] [##############............................................] 8Selecting previously unselected package libmysqlclient20:amd64.
Preparing to unpack .../15-libmysqlclient20_5.7.39-0ubuntu0.18.04.2_amd64.deb ...
Unpacking libmysqlclient20:amd64 (5.7.39-0ubuntu0.18.04.2) ...
7Progress: [ 25%] [##############............................................] 8Selecting previously unselected package qtcore4-l10n.
Preparing to unpack .../16-qtcore4-l10n_4%3a4.8.7+dfsg-7ubuntu1_all.deb ...
7Progress: [ 26%] [###############...........................................] 8Unpacking qtcore4-l10n (4:4.8.7+dfsg-7ubuntu1) ...
7Progress: [ 27%] [###############...........................................] 8Selecting previously unselected package libqtcore4:amd64.
Preparing to unpack .../17-libqtcore4_4%3a4.8.7+dfsg-7ubuntu1_amd64.deb ...
Unpacking libqtcore4:amd64 (4:4.8.7+dfsg-7ubuntu1) ...
7Progress: [ 28%] [################..........................................] 87Progress:

Setting up qtchooser (64-ga1b6736-5) ...
7Progress: [ 63%] [####################################......................] 8Setting up libtiff5:amd64 (4.0.9-5ubuntu0.7) ...
7Progress: [ 65%] [#####################################.....................] 87Progress: [ 65%] [#####################################.....................] 8Setting up libmng2:amd64 (2.0.2-0ubuntu3) ...
7Progress: [ 66%] [######################################....................] 8Setting up libbsd0:amd64 (0.8.7-1ubuntu0.1) ...
7Progress: [ 67%] [######################################....................] 8Setting up libqtcore4:amd64 (4:4.8.7+dfsg-7ubuntu1) ...
7Progress: [ 68%] [#######################################...................] 8Setting up libmysqlclient20:amd64 (5.7.39-0ubuntu0.18.04.2) ...
7Progress: [ 69%] [########################################..................] 8Setting up libxdmcp6:amd64 (1:1.1.2-3) ...
7Progress: [ 70%] [########################################..................]

#### Use this command to troubleshoot a failed pyodbc installation:
!pip install --upgrade --global-option=build_ext --global-option="-I/usr/local/include"  --global-option="-L/usr/local/lib" pyodbc

In [3]:
!pip install pyodbc

     |████████████████████████████████| 464 kB 2.2 MB/s            


In [4]:
!rm /etc/odbcinst.ini
!rm /etc/odbc.ini

In [5]:
!ln -s /tf/odbcinst.ini /etc/odbcinst.ini
!ln -s /tf/odbc.ini /etc/odbc.ini

In [6]:
!cat /tf/odbcinst.ini

[InterSystems ODBC35]
UsageCount=1
Driver=/tf/libirisodbcu35.so
Setup=/tf/libirisodbcu35.so
SQLLevel=1
FileUsage=0
DriverODBCVer=02.10
ConnectFunctions=YYN
APILevel=1
DEBUG=1
CPTimeout=<not pooled>



In [7]:
!cat /tf/odbc.ini

[user]
Driver=InterSystems ODBC35
Protocol=TCP
Host=irisimlsvr
Port=51773
Namespace=USER
UID=SUPERUSER
Password=SYS
Description=Sample namespace
Query Timeout=0
Static Cursors=0



In [8]:
!odbcinst -j

unixODBC 2.3.4
DRIVERS............: /etc/odbcinst.ini
SYSTEM DATA SOURCES: /etc/odbc.ini
FILE DATA SOURCES..: /etc/ODBCDataSources
USER DATA SOURCES..: /root/.odbc.ini
SQLULEN Size.......: 8
SQLLEN Size........: 8
SQLSETPOSIROW Size.: 8


### 2. Verify you see "InterSystems ODBC35" in the drivers list

In [9]:
import pyodbc
print(pyodbc.drivers())

['InterSystems ODBC35']


### 3. Get an ODBC connection 

In [10]:
import pyodbc 
import time


#input("Hit any key to start")
dsn = 'IRIS IntegratedML demo via PyODBC'
server = 'irisimlsvr' #'192.168.99.101' 
port = '1972' #'9091'
database = 'USER' 
username = 'SUPERUSER' 
password = 'SYS' 
cnxn = pyodbc.connect('DRIVER={InterSystems ODBC35};SERVER='+server+';PORT='+port+';DATABASE='+database+';UID='+username+';PWD='+ password)

### Ensure it read strings correctly.
cnxn.setdecoding(pyodbc.SQL_CHAR, encoding='utf8')
cnxn.setdecoding(pyodbc.SQL_WCHAR, encoding='utf8')
cnxn.setencoding(encoding='utf8')

### 4. Get a cursor; start the timer

In [11]:
cursor = cnxn.cursor()
start= time.clock()

### 5. Specify the training data, and give a model name

In [12]:
dataTable = 'Biomedical.BreastCancer'
dataTablePredict = 'Result02'
dataColumn =  'Diagnosis'
dataColumnPredict = "PredictedDiagnosis"
modelName = "bc" #chose a name - must be unique in server end

 ### Cleaning before retrying

In [13]:
#If we re-run the notebook just drop model and table
#cursor.execute("DROP MODEL %s" % modelName)
#cursor.execute("DROP TABLE %s" % dataTablePredict)

### 6. Train and predict

In [14]:
cursor.execute("CREATE MODEL %s PREDICTING (%s)  FROM %s" % (modelName, dataColumn, dataTable))
cursor.execute("TRAIN MODEL %s FROM %s" % (modelName, dataTable))
cursor.execute("Create Table %s (%s VARCHAR(100), %s VARCHAR(100))" % (dataTablePredict, dataColumnPredict, dataColumn))
cursor.execute("INSERT INTO %s  SELECT TOP 20 PREDICT(%s) AS %s, %s FROM %s" % (dataTablePredict, modelName, dataColumnPredict, dataColumn, dataTable)) 
cnxn.commit()

### 7. Show the predict result

In [15]:
import pandas as pd
from IPython.display import display

df1 = pd.read_sql("SELECT * from %s ORDER BY ID" % dataTablePredict, cnxn)
display(df1)

PredictedDiagnosis Diagnosis
0                   M         M
1                   M         M
2                   M         M
3                   M         M
4                   M         M
5                   M         M
6                   M         M
7                   M         M
8                   M         M
9                   M         M
10                  M         M
11                  M         M
12                  M         M
13                  M         M
14                  M         M
15                  M         M
16                  M         M
17                  M         M
18                  M         M
19                  B         B

### 8. Show a complicated query
IntegratedML function PREDICT() and PROBABILITY() can appear virtually anywhere in a SQL query, for maximal flexibility!
Below we are SELECTing columns as well as the result of the PROBABILITY function, and then filtering on the result of the PREDICT function. To top it off, ORDER BY is using the output of PROBSBILITY for sorting.

In [16]:
df2 = pd.read_sql("SELECT ID, PROBABILITY(bc FOR 'M') AS Probability, Diagnosis FROM %s \
                    WHERE Mean_Area BETWEEN 300 AND 600 AND Mean_Radius > 5 AND PREDICT(%s) = 'M' \
                    ORDER BY Probability" % (dataTable, modelName),cnxn)         
display(df2)

ID  Probability diagnosis
0   298         0.62         M
1    41         0.63         M
2   136         0.67         M
3    74         0.68         M
4    42         0.76         M
5   147         0.81         M
6   101         0.82         M
7   194         0.84         M
8    40         0.85         M
9   127         0.85         M
10  380         0.85         M
11   45         0.86         M
12    4         0.88         M
13  216         0.89         M
14  172         0.90         M
15  230         0.92         M
16    6         0.93         M
17   10         0.93         M
18   15         0.93         M
19  106         0.93         M
20   48         0.94         M
21    8         0.97         M
22  197         0.97         M
23  502         0.97         M
24  513         0.98         M
25   44         0.99         M
26    9         1.00         M
27   32         1.00         M
28   65         1.00         M
29  204         1.00         M
30  436         1.00         M

### 9. Close and clean 

In [17]:
cnxn.close()
end= time.clock()
print ("Total elapsed time: ")
print (end-start)
#input("Hit any key to end")

Total elapsed time: 
0.7452100000000002
